In [ ]:
!pip install ezdxf
!pip install opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 32.7 MB/s eta 0:00:00


In [ ]:
#connect to google drive
import os, sys
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import cv2
import numpy as np
import ezdxf
import shutil

def convertDXF(id):

  inpath = '/content/drive/Shareddrives/SmartCLT_DF2023/Scan2/GOOD_SCAN/'
  #inpath1 = '/content/drive/Shareddrives/SmartCLT_DF2023/Scan2/GOOD_SCAN/'
  outputpath = '/content/drive/Shareddrives/SmartCLT_DF2023/T_Algorithm/OUT-DXF/OUTPUT_0628/'

  # Load the image
  image_path = inpath + str(id)+ '_mask.jpg'
  img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

  # Flip the image vertically
  flipped_img = cv2.flip(img, 0)


  # Blur image to enhance edge detection
  flipped_img = cv2.GaussianBlur(flipped_img, (5, 5), 0)

  # Apply Canny edge detection
  edges = cv2.Canny(flipped_img,50,100)

  # Find contours
  contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  # Create a new DXF document.
  doc = ezdxf.new('R2010')
  msp = doc.modelspace()


  # Set the drawing's units to millimeters
  doc.header["$INSUNITS"] = 4
  drawing_units = doc.header["$INSUNITS"]
  print(drawing_units)
  if drawing_units == 4:  # Drawing units are in millimeters
      pixel_to_mm = 1  # 1 pixel = 1 millimeter

  # Set layers
  doc = ezdxf.new(setup=True)  # setup required line types
  msp = doc.modelspace()
  doc.layers.add(name="Curve_Exist", color=63, linetype="Continuous")
  doc.layers.add(name="CuttingCurve_A", color=10, linetype="Continuous")
  doc.layers.add(name="CuttingCurve_B", color=30, linetype="Continuous")
  doc.layers.add(name="Image", color=191, linetype="Continuous")
  doc.layers.add(name="Solution", color=191, linetype="Continuous")


  move_distance = 303
  buffer = 0.025

  # Import background image
  fpath= inpath + str(id) + '_image.jpg'
  fpath1= inpath + str(id)+ '_solution.png'
  my_image_def = doc.add_image_def(filename=fpath, size_in_pixel=(3500/(1-2*buffer), 606))
  my_image_def1 = doc.add_image_def(filename=fpath1, size_in_pixel=(3500, 695))

  # Use shutil.copy() to duplicate the file to the new path
  shutil.copy(fpath, outputpath)
  shutil.copy(fpath1, outputpath)

  msp = doc.modelspace()

  msp.add_image(
      insert=(-move_distance, -3500/(1-2*buffer)*buffer),
      size_in_units=(3500/(1-2*buffer),606),
      image_def=my_image_def,
      rotation=0,
      dxfattribs={"layer": "Image"}
  )

  msp.add_image(
      insert=(-move_distance, 0),
      size_in_units=(3500,695),
      image_def=my_image_def1,
      rotation=0,
      dxfattribs={"layer": "Solution"}
  )


  # Convert contour points to polyline and reverse list
  contour_points = [contour.squeeze().tolist() for contour in contours]  # turn array tuple into a list
  polylst = []
  for i in range(len(contour_points)):
    contour_points2=[[contour_points[i][0][0]-move_distance,contour_points[i][0][1]]]
    ref = 0
    for points in contour_points[i]:
      points = [points[0]-move_distance,points[1]]
      #print(points)
      newpoint = points[1]
      if newpoint > ref:
        contour_points2.append(points)
        ref = newpoint
    print(i,contour_points2)


    polylst.append(contour_points2)
  polylst.reverse()
  print(polylst)

  # Assigning layers
  for i in range(len(polylst)):
    drawpolylst=polylst[i]
    if i==0:
      msp.add_lwpolyline(drawpolylst,dxfattribs={"layer": "Curve_Exist"})
      msp.add_lwpolyline(drawpolylst,dxfattribs={"layer": "CuttingCurve_A"})
    elif i==1:
      msp.add_lwpolyline(drawpolylst,dxfattribs={"layer": "CuttingCurve_B"})
    elif i==2:
      msp.add_lwpolyline(drawpolylst,dxfattribs={"layer": "Curve_Exist"})

  # Save the document.
  outputname = str(id) + '_DF_poly.dxf'
  outputfile = outputpath + outputname
  doc.saveas(outputfile)

In [ ]:
lst = ['22']
#lst = ['03','23','01','21','12','17','19','33','34']
for i in range(len(lst)):
  convertDXF(str(lst[i]))


4
0 [[231, 0], [231, 7], [230, 8], [230, 9], [229, 10], [229, 40], [228, 41], [228, 74], [227, 75], [227, 78], [226, 79], [226, 113], [225, 114], [225, 115], [224, 116], [224, 164], [223, 165], [223, 199], [222, 200], [222, 202], [221, 203], [221, 241], [220, 242], [220, 244], [219, 245], [219, 281], [218, 282], [218, 333], [217, 334], [217, 335], [216, 336], [216, 383], [215, 384], [215, 420], [214, 421], [214, 423], [213, 424], [213, 454], [212, 455], [212, 457], [211, 458], [211, 498], [210, 499], [210, 534], [209, 535], [209, 537], [208, 538], [208, 574], [207, 575], [207, 577], [206, 578], [206, 609], [205, 610], [205, 642], [204, 643], [204, 645], [203, 646], [203, 874], [202, 875], [202, 878], [201, 879], [201, 938], [202, 939], [202, 947], [201, 948], [201, 1027], [202, 1028], [202, 1030], [201, 1031], [201, 1033], [200, 1034], [200, 1169], [201, 1170], [201, 1273], [202, 1274], [202, 1277], [203, 1278], [203, 1321], [204, 1322], [204, 1324], [205, 1325], [205, 1424], [206, 142

# For Test

In [ ]:
      elif i==2:
    msp.add_spline(contour_points2,dxfattribs={"layer": "CuttingCurve_B"})



    if i == 3:
      msp.add_spline(contour_points2,dxfattribs={"layer": "Curve_Exist"})
      contour_points3=[]
      for j in range(len(contour_points2)):
        po=[contour_points2[j][0],contour_points2[j][1]]
        contour_points3.append(po)
      print(contour_points3)
      msp.add_spline(contour_points3,dxfattribs={"layer": "CuttingCurve_A"})
  elif i==2:
    msp.add_spline(contour_points2,dxfattribs={"layer": "CuttingCurve_B"})
  elif i==1:
    msp.add_spline(contour_points2,dxfattribs={"layer": "Curve_Exist"})